In [54]:
import pandas as pd
import numpy as np
df = pd.read_csv('claim.sample.csv', delimiter = ',', na_values=[' '])
#if you go back to the csv file, all missing cells have one space in them, initially I was really buffled by no missing values 

In [55]:
df.shape

(472559, 29)

In [56]:
#data types for each columns and missing data 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472559 entries, 0 to 472558
Data columns (total 29 columns):
V1                           472559 non-null int64
Claim.Number                 472559 non-null float64
Claim.Line.Number            472559 non-null int64
Member.ID                    472559 non-null int64
Provider.ID                  472559 non-null object
Line.Of.Business.ID          265050 non-null float64
Revenue.Code                 472203 non-null object
Service.Code                 472013 non-null object
Place.Of.Service.Code        403 non-null float64
Procedure.Code               296928 non-null object
Diagnosis.Code               472559 non-null object
Claim.Charge.Amount          472559 non-null float64
Denial.Reason.Code           146529 non-null object
Price.Index                  247800 non-null object
In.Out.Of.Network            267637 non-null object
Reference.Index              270462 non-null object
Pricing.Index                270462 non-null object
Capitat

In [57]:
#subset the dataframe to get the J-code claims
df = df[df['Procedure.Code'].notna()]
J_claims_df = df[df['Procedure.Code'].str.startswith('J')]

In [58]:
#number of J-code claims
J_claims_df.shape

(51029, 29)

In [59]:
#generate the labels 
J_claims_df['Provider.Payment.Amount'] = J_claims_df['Provider.Payment.Amount'] > 0 
J_claims_df['Provider.Payment.Amount'] = J_claims_df['Provider.Payment.Amount'].astype(int)

C:\Users\yanli\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\yanli\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [60]:
#checking for missing values
J_claims_df.isnull().sum()

V1                               0
Claim.Number                     0
Claim.Line.Number                0
Member.ID                        0
Provider.ID                      0
Line.Of.Business.ID          18962
Revenue.Code                     2
Service.Code                    59
Place.Of.Service.Code        51027
Procedure.Code                   0
Diagnosis.Code                   0
Claim.Charge.Amount              0
Denial.Reason.Code           40417
Price.Index                  21170
In.Out.Of.Network            18980
Reference.Index              18666
Pricing.Index                18666
Capitation.Index             32331
Subscriber.Payment.Amount        0
Provider.Payment.Amount          0
Group.Index                      0
Subscriber.Index                 0
Subgroup.Index                   0
Claim.Type                       0
Claim.Subscriber.Type            0
Claim.Pre.Prince.Index       39996
Claim.Current.Status             0
Network.ID                   19063
Agreement.ID        

In [61]:
#checking for levels in each columns
for c in J_claims_df.columns:
    print(c, len(J_claims_df[c].unique()))

V1 51029
Claim.Number 10691
Claim.Line.Number 121
Member.ID 6341
Provider.ID 15
Line.Of.Business.ID 6
Revenue.Code 17
Service.Code 12
Place.Of.Service.Code 2
Procedure.Code 202
Diagnosis.Code 1436
Claim.Charge.Amount 3634
Denial.Reason.Code 100
Price.Index 3
In.Out.Of.Network 3
Reference.Index 4
Pricing.Index 4
Capitation.Index 3
Subscriber.Payment.Amount 2
Provider.Payment.Amount 2
Group.Index 781
Subscriber.Index 6243
Subgroup.Index 790
Claim.Type 2
Claim.Subscriber.Type 2
Claim.Pre.Prince.Index 3
Claim.Current.Status 9
Network.ID 15
Agreement.ID 22


In [62]:
#from above, dropping some columns that are likely not useful for the model... my own educated guess here
J_claims_df = J_claims_df.drop(columns = ['V1', 'Claim.Number', 'Claim.Line.Number', 'Member.ID', 'Place.Of.Service.Code', 'Group.Index', 'Subscriber.Index'])

In [63]:
#from the profiling, I will drop Subscriber.Payment.Amount column because there is only one claim has a value different from 0.
J_claims_df = J_claims_df.drop(columns = ['Subscriber.Payment.Amount'])

In [64]:
#check the datatype again
J_claims_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 51029 entries, 3073 to 472463
Data columns (total 21 columns):
Provider.ID                51029 non-null object
Line.Of.Business.ID        32067 non-null float64
Revenue.Code               51027 non-null object
Service.Code               50970 non-null object
Procedure.Code             51029 non-null object
Diagnosis.Code             51029 non-null object
Claim.Charge.Amount        51029 non-null float64
Denial.Reason.Code         10612 non-null object
Price.Index                29859 non-null object
In.Out.Of.Network          32049 non-null object
Reference.Index            32363 non-null object
Pricing.Index              32363 non-null object
Capitation.Index           18698 non-null object
Provider.Payment.Amount    51029 non-null int32
Subgroup.Index             51029 non-null int64
Claim.Type                 51029 non-null object
Claim.Subscriber.Type      51029 non-null object
Claim.Pre.Prince.Index     11033 non-null object
Claim

In [65]:
#Cast the following columns as categorical
for c in ['Line.Of.Business.ID','Subgroup.Index', 'Claim.Current.Status']:
    J_claims_df[c] = J_claims_df[c].astype(str) 

In [66]:
#Impute two columns, Revenue.Code and Service.Code, with their mode 
for column in ['Revenue.Code', 'Service.Code']:
    J_claims_df[column].fillna(J_claims_df[column].mode()[0], inplace=True)

In [67]:
#fill the rest NAs with 'None' as its own category 
J_claims_df.fillna('None', inplace = True)

In [68]:
#check the df for missing value again
J_claims_df.isnull().sum()

Provider.ID                0
Line.Of.Business.ID        0
Revenue.Code               0
Service.Code               0
Procedure.Code             0
Diagnosis.Code             0
Claim.Charge.Amount        0
Denial.Reason.Code         0
Price.Index                0
In.Out.Of.Network          0
Reference.Index            0
Pricing.Index              0
Capitation.Index           0
Provider.Payment.Amount    0
Subgroup.Index             0
Claim.Type                 0
Claim.Subscriber.Type      0
Claim.Pre.Prince.Index     0
Claim.Current.Status       0
Network.ID                 0
Agreement.ID               0
dtype: int64

In [69]:
#checking for levels in each columns
for c in J_claims_df.columns:
    print(c, len(J_claims_df[c].unique()))

Provider.ID 15
Line.Of.Business.ID 6
Revenue.Code 16
Service.Code 11
Procedure.Code 202
Diagnosis.Code 1436
Claim.Charge.Amount 3634
Denial.Reason.Code 100
Price.Index 3
In.Out.Of.Network 3
Reference.Index 4
Pricing.Index 4
Capitation.Index 3
Provider.Payment.Amount 2
Subgroup.Index 790
Claim.Type 2
Claim.Subscriber.Type 2
Claim.Pre.Prince.Index 3
Claim.Current.Status 9
Network.ID 15
Agreement.ID 22


In [70]:
J_claims_df.shape

(51029, 21)

In [71]:
#labels
labels = J_claims_df['Provider.Payment.Amount']

In [72]:
#one hot encoding diagnosis code
DC_encoded = pd.get_dummies(J_claims_df['Diagnosis.Code'], prefix = 'DC')
DC_encoded = DC_encoded[['DC_V5811', 'DC_5856', 'DC_29689', 'DC_64403', 'DC_27900', 'DC_5559',
       'DC_64893', 'DC_29653', 'DC_28731', 'DC_34690', 'DC_27903', 'DC_5552',
       'DC_2859', 'DC_65983', 'DC_78701', 'DC_7840', 'DC_35800', 'DC_99675',
       'DC_7212', 'DC_6823', 'DC_23873', 'DC_81305', 'DC_40301', 'DC_V5812',
       'DC_59010', 'DC_25002', 'DC_4660', 'DC_7323', 'DC_2727', 'DC_33819',
       'DC_7140', 'DC_20400', 'DC_78703', 'DC_7880', 'DC_6929', 'DC_2853',
       'DC_29680', 'DC_99741', 'DC_64413', 'DC_20280', 'DC_5550', 'DC_V7283',
       'DC_78909', 'DC_64303', 'DC_66624', 'DC_55090', 'DC_7291', 'DC_78060',
       'DC_64623', 'DC_38611', 'DC_53530', 'DC_56942', 'DC_6185', 'DC_64233',
       'DC_8363', 'DC_V2889', 'DC_64193', 'DC_1628', 'DC_2809', 'DC_7455',
       'DC_6930', 'DC_81301', 'DC_9110', 'DC_71535', 'DC_5566', 'DC_37487',
       'DC_8449', 'DC_481', 'DC_29643', 'DC_64683', 'DC_V7651', 'DC_73018',
       'DC_80701', 'DC_56032', 'DC_1820', 'DC_78904', 'DC_4610', 'DC_64663',
       'DC_49122', 'DC_340', 'DC_7866', 'DC_73342', 'DC_68100', 'DC_7592',
       'DC_V571', 'DC_49392', 'DC_37400', 'DC_99889', 'DC_78900', 'DC_34580',
       'DC_37433', 'DC_3419', 'DC_7804', 'DC_47819', 'DC_71940', 'DC_6089',
       'DC_25000', 'DC_83200', 'DC_35781', 'DC_78606']]

In [73]:
DC_encoded.shape

(51029, 100)

In [74]:
#drop subgroup.index and dignosis code, labels  then one hot encoding
J_claims_df = J_claims_df.drop(columns = ['Subgroup.Index', 'Diagnosis.Code', 'Provider.Payment.Amount'])
train_dummy = pd.get_dummies(J_claims_df)

#concat the top 100 features from diagnosis code, data-train
train = pd.concat([train_dummy, DC_encoded], axis = 1)

In [75]:
train.shape

(51029, 521)

In [76]:
#train.to_csv('J_claims.csv', index = False)

In [77]:
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

#split into train and valid datset
X_train, X_valid, y_train, y_valid = train_test_split(train, labels, test_size=0.10, random_state=101)

#train a model
rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train,y_train)

#Variable importance table
feature_imp = pd.Series(rf.feature_importances_,index=X_train.columns).sort_values(ascending=False)

#make prediction of test dataset
predictions_randf = rf.predict(X_valid)
auc = roc_auc_score(y_valid, predictions_randf)
auc

0.9736490605546801

In [78]:
F1score = f1_score(y_valid, predictions_randf)
F1score

0.9503784693019344

In [79]:
Accscore = accuracy_score(y_valid, predictions_randf)
Accscore

0.9884381736233588

In [80]:
labels[120:140]

3653    0
3654    0
3655    0
3675    0
3686    0
3687    0
3735    0
3737    0
3738    0
3739    0
3740    0
3741    0
3742    0
3743    0
3744    0
3769    1
3801    1
3807    1
3840    0
3844    0
Name: Provider.Payment.Amount, dtype: int32

In [84]:
train

,Claim.Charge.Amount,Provider.ID_FA0001387001,Provider.ID_FA0001387002,Provider.ID_FA0001389001,Provider.ID_FA0001389003,Provider.ID_FA0001411001,Provider.ID_FA0001411003,Provider.ID_FA0001774001,Provider.ID_FA0001774002,Provider.ID_FA0004551001,...,DC_37433,DC_3419,DC_7804,DC_47819,DC_71940,DC_6089,DC_25000,DC_83200,DC_35781,DC_78606
3073,592.201995,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3091,118.832970,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3092,196.890405,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3103,592.201995,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3114,592.201995,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3123,592.201995,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3138,592.201995,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3150,592.201995,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3168,122.017980,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3170,357.597615,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [85]:
from xgboost import XGBClassifier

#split into train and valid datset
X_train, X_valid, y_train, y_valid = train_test_split(train, labels, test_size=0.10, random_state=101)

#train a model
xgb = XGBClassifier(n_estimators=100)
xgb.fit(X_train,y_train)

#Variable importance table
feature_imp = pd.Series(xgb.feature_importances_,index=X_train.columns).sort_values(ascending=False)

#make prediction of test dataset
predictions_xgb = xgb.predict(X_valid)
auc = roc_auc_score(y_valid, predictions_xgb)
auc

0.973052732959073